<a href="https://colab.research.google.com/github/Scarfaced007/CNN-DL/blob/main/DL_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False)

100%|██████████| 170M/170M [00:05<00:00, 30.9MB/s]


In [3]:
model = torchvision.models.resnet18(weights=None)

model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(512, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

In [ ]:
epochs = 25
history = {'train_loss': [], 'train_acc': [], 'test_acc': []}

for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    scheduler.step()

    train_acc = 100. * correct / total
    history['train_loss'].append(running_loss / len(train_loader))
    history['train_acc'].append(train_acc)

    model.eval()
    test_correct, test_total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    test_acc = 100. * test_correct / test_total
    history['test_acc'].append(test_acc)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {history['train_loss'][-1]:.4f} | "f"Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%")

Epoch 1/25 | Loss: 2.0121 | Train Acc: 29.57% | Test Acc: 43.63%
Epoch 2/25 | Loss: 1.3156 | Train Acc: 52.18% | Test Acc: 54.72%
Epoch 3/25 | Loss: 0.9797 | Train Acc: 65.22% | Test Acc: 64.31%
Epoch 4/25 | Loss: 0.7531 | Train Acc: 73.29% | Test Acc: 72.54%
Epoch 5/25 | Loss: 0.5943 | Train Acc: 79.35% | Test Acc: 74.63%
Epoch 6/25 | Loss: 0.4885 | Train Acc: 82.99% | Test Acc: 75.00%
Epoch 7/25 | Loss: 0.4165 | Train Acc: 85.50% | Test Acc: 78.09%
Epoch 8/25 | Loss: 0.3669 | Train Acc: 87.30% | Test Acc: 78.99%
Epoch 9/25 | Loss: 0.3195 | Train Acc: 88.87% | Test Acc: 78.73%
Epoch 10/25 | Loss: 0.2943 | Train Acc: 89.80% | Test Acc: 79.65%
Epoch 11/25 | Loss: 0.2542 | Train Acc: 91.19% | Test Acc: 79.74%
Epoch 12/25 | Loss: 0.2378 | Train Acc: 91.70% | Test Acc: 76.24%
Epoch 13/25 | Loss: 0.2093 | Train Acc: 92.70% | Test Acc: 77.30%


In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.title('Baseline: Training Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['test_acc'], label='Test Acc')
plt.title('Baseline: Accuracy Curves')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()

In [ ]:
import torch.nn.functional as F
import numpy as np

def get_high_confidence_failures(model, loader, threshold=0.85, num_images=5):
    """
    Finds images where the model is incorrect but predicts with high confidence.
    """
    model.eval()
    failures = []
    classes = ('Plane', 'Car', 'Bird', 'Cat', 'Deer',
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

    # Normalization constants (to un-normalize images for plotting)
    mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1).to(device)
    std = torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1).to(device)

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Get probabilities (Softmax)
            probs = F.softmax(outputs, dim=1)
            conf, preds = torch.max(probs, dim=1)

            # Find errors: Prediction != Label AND Confidence > Threshold
            wrong_indices = (preds != labels) & (conf > threshold)
            failure_idx = torch.nonzero(wrong_indices, as_tuple=True)[0]

            if len(failure_idx) > 0:
                for idx in failure_idx:
                    if len(failures) >= num_images:
                        return failures

                    # Un-normalize image for visualization
                    img = inputs[idx] * std + mean
                    img = torch.clamp(img, 0, 1) # Ensure pixel values are valid [0,1]

                    failures.append({
                        'image': img.cpu(),
                        'true': classes[labels[idx].item()],
                        'pred': classes[preds[idx].item()],
                        'conf': conf[idx].item()
                    })
    return failures

def plot_failures(failures):
    """
    Plots the collected failure cases.
    """
    plt.figure(figsize=(15, 4))
    for i, fail in enumerate(failures):
        ax = plt.subplot(1, len(failures), i + 1)

        # Convert tensor (C, H, W) -> numpy (H, W, C)
        img_np = fail['image'].permute(1, 2, 0).numpy()

        plt.imshow(img_np)
        plt.title(f"True: {fail['true']}\nPred: {fail['pred']}\nConf: {fail['conf']:.2f}",
                  color='red', fontsize=12)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# --- Execution ---
print("Searching for high-confidence failures...")
# Note: If no failures are found, try lowering the threshold (e.g., to 0.80)
failure_cases = get_high_confidence_failures(model, test_loader, threshold=0.90, num_images=5)

if len(failure_cases) > 0:
    print(f"Found {len(failure_cases)} high-confidence failures:")
    plot_failures(failure_cases)
else:
    print("No failures found above the threshold. Your model might be too accurate or calibrated!")